# 1) Data and package Imports

In [56]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [57]:
%cd /content/drive/My Drive/Fairmatic

/content/drive/My Drive/Fairmatic


In [58]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

In [59]:
df = pd.read_excel('data.xlsx', sheet_name='Raw Data')
df.head()

,Opportunity Name,Stage,Submission Received Date,Quote Sent Date,Close Date,Annual Premium,Model Segment,Declined Reason,Lost Reason,Declined Other Reason,Lost Other Reason
0,Opportunity 1,Declined,1/1/2023,NaN,4/1/2023,50000.0,Mid-Market,Business Class,NaN,Declined due to operations not being an artisa...,NaN
1,Opportunity 2,Declined,1/1/2023,NaN,4/1/2023,122500.0,Mid-Market,Vehicle weight,NaN,heavy vehicles and ineligible trailers,NaN
2,Opportunity 3,Declined,1/10/2023,NaN,2/15/2023,24500.0,Mid-Market,Other,NaN,"Only 7 units on schedule, our min for this seg...",NaN
3,Opportunity 4,Declined,1/10/2023,NaN,2/10/2023,50000.0,NEMT,Business Class,NaN,"Level of patient care provided. In addition, t...",NaN
4,Opportunity 5,Declined,1/10/2023,NaN,1/23/2023,80500.0,Mid-Market,Business Class,NaN,Not an Artisan class of business. (Storage & M...,NaN


In [60]:
df['Annual Premium(Thousands)'] = df['Annual Premium']/10000

# 2) Data Inspection

## a) Data Summary

In [61]:
df.groupby('Stage').agg({'Opportunity Name':'count','Annual Premium(Thousands)':'sum'}).sort_values(by='Opportunity Name', ascending=False)

,Opportunity Name,Annual Premium(Thousands)
Stage,,
Declined,4780,59211.612647
Lost,1002,15504.803693
Won,184,3636.429154
Quote Sent,29,275.527140
Indication Sent,7,45.200000
Opportunity Created,1,7.500000


In [62]:
df.groupby('Declined Reason').agg({'Opportunity Name':'count', 'Annual Premium(Thousands)':'sum'}).sort_values(by='Opportunity Name', ascending=False)

,Opportunity Name,Annual Premium(Thousands)
Declined Reason,,
Business Class,1337,17155.707300
Other,962,11885.200778
Operating state,472,6617.134400
Unresponsive Broker,435,3957.240545
Fleet size below min unit count,318,329.484100
...,...,...
Operating state; Pricing not competitive; Other,1,15.000000
Business Class; Operating state; Vehicle type; Vehicle weight; Vehicle age,1,15.400000
Operating state; Insufficient Lead Time,1,294.700000


In [63]:
df.groupby('Declined Other Reason').agg({'Opportunity Name':'count', 'Annual Premium(Thousands)':'sum'}).sort_values(by='Opportunity Name', ascending=False)

,Opportunity Name,Annual Premium(Thousands)
Declined Other Reason,,
"It was in stale state, before V1.0 Powertrain. Hence marking it as Declined.",66,972.25475
Amazon DSP - past effective date / no admitted solution in place,61,810.25000
Amazon DSP,44,606.65000
FL,33,643.51700
GC,15,152.69360
...,...,...
"Declined due to operations of Portable Toilet Rentals, septic tank and grease trap services not being an artisan class for our carrier partner.",1,6.65000
Declined due to operations not falling under an artisan risk,1,5.00000
Declined due to operations not being an artisan risk.,1,5.00000


## b) Data Cleaning

+ Check duplicated data
+ Check null data
+ Check inconsistencies data
+ Check

#### i) Check duplicate

In [64]:
df[df.duplicated()]

,Opportunity Name,Stage,Submission Received Date,Quote Sent Date,Close Date,Annual Premium,Model Segment,Declined Reason,Lost Reason,Declined Other Reason,Lost Other Reason,Annual Premium(Thousands)


No duplication

#### ii) Check null

In [65]:
null_percent = (df.isnull().sum() / len(df)) * 100
print(null_percent)

Opportunity Name              0.000000
Stage                         0.000000
Submission Received Date      0.000000
Quote Sent Date              79.493587
Close Date                    0.000000
Annual Premium                0.000000
Model Segment                 0.000000
Declined Reason              17.091454
Lost Reason                  82.608696
Declined Other Reason        56.605031
Lost Other Reason            92.786940
Annual Premium(Thousands)     0.000000
dtype: float64


<u>Hypothesis I:</u>

Quote sent ~79% --> approximate 21% sub-to-quote ratio

Declined 17% --> 83 % to quote != 79 % --> need to investigate on overlapped
data or data inconsistencies (conflicting but may have some submission not being declined but still not being able to turn to quote ?)

<u>Hypothesis II:</u>

Lost ~ 18% of all submission + 17% of non-declined --> won ~ -1 % --> Data inconsistencies exists  







#### iii) Data Inconsistencies

#### Date inconsistencies

In [66]:
#change datetime column
df['Submission Received Date'] = pd.to_datetime(df['Submission Received Date'])
df['Quote Sent Date'] = pd.to_datetime(df['Quote Sent Date'])
df['Close Date'] = pd.to_datetime(df['Close Date'])

df[['Submission Received Date', 'Quote Sent Date','Close Date']].dtypes

Submission Received Date    datetime64[ns]
Quote Sent Date             datetime64[ns]
Close Date                  datetime64[ns]
dtype: object

In [67]:
df[(df['Submission Received Date'] > df['Quote Sent Date'])]

,Opportunity Name,Stage,Submission Received Date,Quote Sent Date,Close Date,Annual Premium,Model Segment,Declined Reason,Lost Reason,Declined Other Reason,Lost Other Reason,Annual Premium(Thousands)
889,Opportunity 890,Won,2023-10-19,2023-09-21,2023-10-19,149125.94,Last Mile,NaN,NaN,NaN,NaN,14.912594
1887,Opportunity 1888,Quote Sent,2023-12-13,2023-09-14,2023-10-01,90636.70,Last Mile,NaN,NaN,NaN,NaN,9.063670
2014,Opportunity 2015,Declined,2023-12-19,2023-07-12,2024-01-31,100000.00,Mid-Market,Pricing not competitive,Other,NaN,"It was in stale state, before V1.0 Powertrain....",10.000000
5661,Opportunity 5662,Won,2023-09-18,2023-09-01,2023-09-11,494342.00,Mid-Market,NaN,NaN,NaN,NaN,49.434200
5896,Opportunity 5897,Won,2023-09-29,2023-09-28,2023-09-29,120000.00,TNC,NaN,NaN,NaN,NaN,12.000000


In [68]:
# Extract correct data
df_quote = df[(df['Quote Sent Date'].notnull()) & (df['Submission Received Date'] < df['Quote Sent Date'])]

# Mean quote sent
df_quote['datediff'] = df_quote['Quote Sent Date'] - df_quote['Submission Received Date']
mean_quote_sent = df_quote['datediff'].mean()

# Replace data
mask = (df['Submission Received Date'] > df['Quote Sent Date'])
df.loc[mask, 'Quote Sent Date'] = df.loc[mask, 'Submission Received Date'] + mean_quote_sent
df.loc[mask, 'Quote Sent Date'] = df.loc[mask, 'Quote Sent Date'].dt.strftime('%Y-%m-%d')

# Display
df[mask]

<ipython-input-68-d1c2366ecd0d>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_quote['datediff'] = df_quote['Quote Sent Date'] - df_quote['Submission Received Date']


,Opportunity Name,Stage,Submission Received Date,Quote Sent Date,Close Date,Annual Premium,Model Segment,Declined Reason,Lost Reason,Declined Other Reason,Lost Other Reason,Annual Premium(Thousands)
889,Opportunity 890,Won,2023-10-19,2023-11-07,2023-10-19,149125.94,Last Mile,NaN,NaN,NaN,NaN,14.912594
1887,Opportunity 1888,Quote Sent,2023-12-13,2024-01-01,2023-10-01,90636.70,Last Mile,NaN,NaN,NaN,NaN,9.063670
2014,Opportunity 2015,Declined,2023-12-19,2024-01-07,2024-01-31,100000.00,Mid-Market,Pricing not competitive,Other,NaN,"It was in stale state, before V1.0 Powertrain....",10.000000
5661,Opportunity 5662,Won,2023-09-18,2023-10-07,2023-09-11,494342.00,Mid-Market,NaN,NaN,NaN,NaN,49.434200
5896,Opportunity 5897,Won,2023-09-29,2023-10-18,2023-09-29,120000.00,TNC,NaN,NaN,NaN,NaN,12.000000


In [69]:
df[(df['Submission Received Date'] > df['Quote Sent Date'])]

,Opportunity Name,Stage,Submission Received Date,Quote Sent Date,Close Date,Annual Premium,Model Segment,Declined Reason,Lost Reason,Declined Other Reason,Lost Other Reason,Annual Premium(Thousands)


Suppose the typical duration between receiving a submission and sending a quote is n days. We will replace all the correct data entries by adding n days to the corresponding "Submission Received Date" value.

We will do the same with incorrect closed date

In [70]:
df[df['Submission Received Date'] > df['Close Date']].head()

,Opportunity Name,Stage,Submission Received Date,Quote Sent Date,Close Date,Annual Premium,Model Segment,Declined Reason,Lost Reason,Declined Other Reason,Lost Other Reason,Annual Premium(Thousands)
517,Opportunity 518,Declined,2024-01-30,NaT,2024-01-02,3500.0,Last Mile,Business Class; Fleet size below min unit count,NaN,NaN,NaN,0.35000
987,Opportunity 988,Declined,2023-10-24,NaT,2023-10-21,59500.0,Mid-Market,Business Class; Fleet size below min unit count,NaN,NaN,NaN,5.95000
1887,Opportunity 1888,Quote Sent,2023-12-13,2024-01-01,2023-10-01,90636.7,Last Mile,NaN,NaN,NaN,NaN,9.06367
1995,Opportunity 1996,Declined,2023-12-18,NaT,2023-12-08,49000.0,Mid-Market,Pricing not competitive,NaN,NaN,NaN,4.90000
2614,Opportunity 2615,Declined,2024-02-19,NaT,2024-02-07,36000.0,Mid-Market,Insufficient Lead Time,NaN,NaN,NaN,3.60000


In [71]:
# Extract correct data
df_close = df[(df['Close Date'].notnull()) & (df['Submission Received Date'] < df['Close Date'])]

# Mean close date
df_close['datediff'] = df_close['Close Date'] - df_close['Submission Received Date']
mean_close_date = df_quote['datediff'].mean().days

# Replace data
mask = (df['Submission Received Date'] > df['Close Date'])
df.loc[mask, 'Close Date'] = df.loc[mask, 'Submission Received Date'] + pd.Timedelta(days=mean_close_date)

# Modify "Submission Received Date" and "Close Date" columns to display year-month-day format
df['Submission Received Date'] = df['Submission Received Date'].dt.strftime('%Y-%m-%d')
df['Close Date'] = df['Close Date'].dt.strftime('%Y-%m-%d')

<ipython-input-71-c032c3429ef8>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_close['datediff'] = df_close['Close Date'] - df_close['Submission Received Date']


In [72]:
df[df['Submission Received Date'] > df['Close Date']].head()

,Opportunity Name,Stage,Submission Received Date,Quote Sent Date,Close Date,Annual Premium,Model Segment,Declined Reason,Lost Reason,Declined Other Reason,Lost Other Reason,Annual Premium(Thousands)


#### Fact Inconsistencies

In [73]:
df[(df['Stage']=='Declined') & (df['Quote Sent Date'].notnull())]

,Opportunity Name,Stage,Submission Received Date,Quote Sent Date,Close Date,Annual Premium,Model Segment,Declined Reason,Lost Reason,Declined Other Reason,Lost Other Reason,Annual Premium(Thousands)
392,Opportunity 393,Declined,2024-01-25,2024-02-26,2024-03-08,820300.00,NEMT,Other,NaN,BOR'd to other agent.,NaN,82.030000
474,Opportunity 475,Declined,2023-01-03,2023-01-17,2023-01-29,42000.00,Mid-Market,Pricing not competitive,NaN,NaN,NaN,4.200000
909,Opportunity 910,Declined,2022-10-20,2022-11-01,2023-02-01,82156.00,Cannabis,Prior Submission,NaN,NaN,NaN,8.215600
1639,Opportunity 1640,Declined,2022-11-03,2022-11-23,2023-01-01,340108.95,Mid-Market,Unresponsive Broker,NaN,NaN,NaN,34.010895
2014,Opportunity 2015,Declined,2023-12-19,2024-01-07,2024-01-31,100000.00,Mid-Market,Pricing not competitive,Other,NaN,"It was in stale state, before V1.0 Powertrain....",10.000000
2055,Opportunity 2056,Declined,2022-12-20,2023-01-08,2023-01-31,150000.00,Mid-Market,Unresponsive Broker,NaN,NaN,NaN,15.000000
2206,Opportunity 2207,Declined,2022-12-29,2023-01-10,2023-01-29,100000.00,TNC,Unresponsive Broker,NaN,NaN,NaN,10.000000
2274,Opportunity 2275,Declined,2022-12-06,2023-01-10,2023-01-21,100000.00,Last Mile,Other,NaN,Amazon DSP,NaN,10.000000
2399,Opportunity 2400,Declined,2022-12-09,2022-12-22,2023-01-15,11984.00,Mid-Market,Operating state,NaN,Operates in Florida,NaN,1.198400
3298,Opportunity 3299,Declined,2023-03-21,2023-04-01,2023-04-01,231000.00,Mid-Market,Operating state,NaN,KS,NaN,23.100000


Submission has been declined but still sent quote --> Decline after the quote being sent ?

1 lost reason (op 2015): machine not compatible

Declined Reasons: Unresponsive workers, Pricing Not Competitive, Operating Sate, Business Class .

In [74]:
df[(df['Stage']=='Won') & (df['Lost Reason'].notnull())]

,Opportunity Name,Stage,Submission Received Date,Quote Sent Date,Close Date,Annual Premium,Model Segment,Declined Reason,Lost Reason,Declined Other Reason,Lost Other Reason,Annual Premium(Thousands)
1997,Opportunity 1998,Won,2022-12-19,2023-04-14,2023-11-01,150000.0,TNC,NaN,Other,NaN,NaN,15.0000
2804,Opportunity 2805,Won,2024-02-24,2024-03-13,2024-02-29,53046.0,Last Mile,NaN,Unresponsive,NaN,never heard back from broker,5.3046
4241,Opportunity 4242,Won,2023-06-13,2023-07-26,2023-08-08,58710.0,Mid-Market,NaN,Other,NaN,"It was in stale state, before V1.0 Powertrain....",5.8710


In [75]:
df[(df['Stage']=='Won') & (df['Lost Reason'].notnull())]['Stage'] = 'Lost'

<ipython-input-75-f4ea926854a5>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[(df['Stage']=='Won') & (df['Lost Reason'].notnull())]['Stage'] = 'Lost'


In [76]:
df[(df['Stage']=='Won') & (df['Lost Reason'].notnull())]

,Opportunity Name,Stage,Submission Received Date,Quote Sent Date,Close Date,Annual Premium,Model Segment,Declined Reason,Lost Reason,Declined Other Reason,Lost Other Reason,Annual Premium(Thousands)
1997,Opportunity 1998,Won,2022-12-19,2023-04-14,2023-11-01,150000.0,TNC,NaN,Other,NaN,NaN,15.0000
2804,Opportunity 2805,Won,2024-02-24,2024-03-13,2024-02-29,53046.0,Last Mile,NaN,Unresponsive,NaN,never heard back from broker,5.3046
4241,Opportunity 4242,Won,2023-06-13,2023-07-26,2023-08-08,58710.0,Mid-Market,NaN,Other,NaN,"It was in stale state, before V1.0 Powertrain....",5.8710


In [77]:
#Has been closed (final result is declined) but still being segmented as "Opportunity Created"
df.loc[(df['Stage']=='Opportunity Created')] = "Declined"

#### Data Incompleteness

In [78]:
df[df['Submission Received Date']=='Declined']

,Opportunity Name,Stage,Submission Received Date,Quote Sent Date,Close Date,Annual Premium,Model Segment,Declined Reason,Lost Reason,Declined Other Reason,Lost Other Reason,Annual Premium(Thousands)
1973,Declined,Declined,Declined,Declined,Declined,Declined,Declined,Declined,Declined,Declined,Declined,Declined


Invalid row --> drop row

In [79]:
df = df[df['Submission Received Date'] != 'Declined']

In [80]:
df['Submission Received Date'].sort_values(ascending=False).head(1)

3672    2024-04-18
Name: Submission Received Date, dtype: object

In [89]:
df['Close Date'] = pd.to_datetime(df['Close Date'])

df_sep = df[(df['Close Date'].dt.year == 2023) & (df['Close Date'].dt.month == 9)][['Declined Reason','Declined Other Reason']]

In [92]:
df_sep = df_sep[df_sep['Declined Reason'].notnull()]

In [94]:
df_sep['Declined Reason'].value_counts()

Declined Reason
Business Class                                                                    114
Other                                                                              85
Fleet size below min unit count                                                    66
Operating state                                                                    61
Unresponsive Broker                                                                42
Below Minimum Premium                                                              41
Below Minimum Premium; Fleet size below min unit count                             13
Vehicle weight                                                                     13
Pricing not competitive                                                             8
Business Class; Vehicle weight                                                      6
Operating state; Other                                                              6
Prior Submission                      

Only 2 submissions received in the most recent month

In [ ]:
df['Submission Received Date'].sort_values(ascending=True).head(1)

In [ ]:
df[(df['Submission Received Date'].dt.year == 2021)]

First opportunity received in the company

# 3) Decline Analysis

### a) All declined reasons by quantity

In [ ]:
df_declined = df.dropna(subset=["Declined Reason"])

In [ ]:
filtered_df = df_declined[df_declined["Declined Reason"].str.contains("Business Class", case=False)]

num_rows = len(filtered_df)
num_rows = filtered_df.shape[0]

num_rows

In [ ]:
df_declined['Declined Reason'] = df_declined['Declined Reason'].str.split("; ")

In [ ]:
big_list = sum([sublist * len(sublist) for sublist in df_declined['Declined Reason']], [])

len(big_list)

In [ ]:
declined_series = pd.Series(big_list)
declined_series = pd.Series(declined_series.value_counts())

declined_series.to_csv("declined.csv")

In [ ]:
fig, ax = plt.subplots(figsize=(20, 10))

sns.barplot(declined_series, ax=ax)

# Rotate the x-axis tick labels to vertical
plt.xticks(rotation=90)
plt.show()

In [ ]:
declined_series

### b) Declined Reasons specific by real-time analysis

In [ ]:
#all the reasons for analyzing
reasons = ['Business Class', 'Other', 'Operating state', 'Vehicle weight', 'Unresponsive Broker', 'Fleet size below min unit count', 'Vehicle type', 'Pricing not competitive', 'Vehicle age', 'Prior Submission', 'Below Minimum Premium', 'Insufficient Lead Time', 'Loss ratio', 'Technology issue', 'Refused to provide info']

# Create a figure with 5 rows and 3 columns of subplots
fig, axes = plt.subplots(5, 3, figsize=(30, 20))

# Flatten the axes array to iterate over each subplot
axes = axes.flatten()

# Iterate over each reason and create a subplot
for i, reason in enumerate(reasons):
    bc_df = df_declined[df_declined['Declined Reason'].apply(lambda x: reason in x)]
    bc_df['Submission Received Date'] = pd.to_datetime(bc_df['Submission Received Date'])

    month_counts = bc_df.groupby([bc_df['Submission Received Date'].dt.year, bc_df['Submission Received Date'].dt.month]).size()

    # Sort the months in alphabetical order
    month_counts = month_counts.reindex(sorted(month_counts.index))

    # Create the bar plot in the current subplot
    month_counts.plot(kind='bar', ax=axes[i])
    axes[i].set_title(reason)
    axes[i].tick_params(axis='x', rotation=45)

# Adjust the spacing between subplots
plt.subplots_adjust(hspace=0.5)

# Show the plot
plt.show()

Still increase dramatically in last year, but downward this year.

# 4) Date diff vs closed case

In [ ]:
qs_df = df[df['Quote Sent Date'].notnull()]

qs_df['Submission Received Date'] = pd.to_datetime(qs_df['Submission Received Date'])
qs_df['Quote Sent Date'] = pd.to_datetime(qs_df['Quote Sent Date'])

In [ ]:
qs_df['Date Process'] = qs_df['Quote Sent Date'] - qs_df['Submission Received Date']

qs_df.head()

In [ ]:
print("Average Decline Time:")
print(qs_df[qs_df['Stage']=='Declined']['Date Process'].mean())
print("Average Quote Time:")
print(qs_df[qs_df['Stage']!='Declined']['Date Process'].mean())

In [ ]:
df[df['Opportunity Name']=='Opportunity 11']

In [ ]:
# Create a figure with two subplots
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# Create the first boxplot for declined submissions
sns.boxplot(qs_df[qs_df['Stage']=='Declined']['Date Process'], ax=ax1)
ax1.set_title('Declined Submissions')
ax1.set_xlabel('Date Process')

# Create the second boxplot for non-declined submissions
sns.boxplot(qs_df[qs_df['Stage']!='Declined']['Date Process'], ax=ax2)
ax2.set_title('Non-Declined Submissions')
ax2.set_xlabel('Date Process')

# Adjust the spacing between subplots
plt.subplots_adjust(wspace=0.5)

# Show the plot
plt.show()

# 5) Declined by Other Reasons

### a) Biagrams & Triagrams

In [95]:
import nltk
nltk.download('punkt')

from nltk import ngrams

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [96]:
df_declined_others = df_declined[df_declined['Declined Other Reason'].notnull()]

In [97]:
declined_other_reasons = df_declined_others['Declined Other Reason']

In [98]:
tokenized_reasons = [nltk.word_tokenize(reason) for reason in declined_other_reasons]

# List of words to exclude
words_to_exclude = ['due', 'to']

# Filter out the words to exclude from the tokenized text
filtered_reasons = [[word for word in tokens if word.lower() not in words_to_exclude] for tokens in tokenized_reasons]

In [99]:
bigrams = [list(ngrams(tokens, 2)) for tokens in filtered_reasons]
trigrams = [list(ngrams(tokens, 3)) for tokens in filtered_reasons]

In [100]:
from nltk import FreqDist

bigram_freq = FreqDist([gram for sublist in bigrams for gram in sublist])
trigram_freq = FreqDist([gram for sublist in trigrams for gram in sublist])

In [101]:
# Get the top 10 bigrams and trigrams
top_10_bigrams = bigram_freq.most_common(10)
top_10_trigrams = trigram_freq.most_common(10)

# Create Pandas DataFrames
bigram_df = pd.DataFrame(top_10_bigrams, columns=['Bigram', 'Frequency'])
trigram_df = pd.DataFrame(top_10_trigrams, columns=['Trigram', 'Frequency'])

# Display the DataFrames
print("Top 10 Bigrams:")
print(bigram_df)
print("\nTop 10 Trigrams:")
print(trigram_df)

Top 10 Bigrams:
                 Bigram  Frequency
0         (Amazon, DSP)        186
1     (effective, date)        182
2  (admitted, solution)        133
3         (Closed, out)        126
4        (no, admitted)        115
5           (in, place)        113
6        (solution, in)        109
7     (past, effective)        104
8              (-, not)        104
9         (an, artisan)         98

Top 10 Trigrams:
                    Trigram  Frequency
0  (no, admitted, solution)        113
1  (admitted, solution, in)        109
2     (solution, in, place)        109
3   (past, effective, date)        104
4          (Amazon, DSP, -)         96
5      (-, past, effective)         89
6         (/, no, admitted)         86
7      (effective, date, /)         81
8             (date, /, no)         80
9            (DSP, -, past)         79


### b) Top 10 frequency investiagting

In [ ]:
df_declined = df.dropna(subset=["Declined Reason"])

In [ ]:
df_declined[df_declined['Declined Reason'].str.contains("Amazon")]

In [ ]:
df_declined